In [86]:
import sys
import os
from datetime import datetime
from google.oauth2 import service_account
from googleapiclient import discovery
from googleapiclient.http import MediaFileUpload
from datetime import datetime
import pandas as pd
import numpy as np
import time
import pyodbc
print(os.path.dirname(sys.executable))

d:\data_sale\venv\Scripts


In [87]:
server = '115.165.164.234'
driver = 'SQL Server'
db1 = 'PhaNam_eSales_PRO'
tcon = 'no'
uname = 'duyvq'
pword = '123VanQuangDuy'
cnxn = pyodbc.connect(driver='{ODBC Driver 17 for SQL Server}', 
                      host=server, database=db1, trusted_connection=tcon,
                      user=uname, password=pword)
cursor = cnxn.cursor()

In [88]:
datenow = datetime.now().strftime("%Y%m%d")
param_1 = f"'20161213'"
param_2 = f"'{datenow}'"

In [89]:
query = f"EXEC [pr_AR_TrackingDebtConfirm]  @Fromdate={param_1}, @Todate={param_2}"
start=time.time()
df1 = pd.read_sql(query, cnxn, parse_dates=['DateOfOrder'])
df1.to_csv('doanhthutienmat_data1.csv', index = False)
end=time.time()
duration=end-start
print(duration)
cnxn.close()

10.027944087982178


In [90]:
df1 = pd.read_csv('doanhthutienmat_data1.csv', dayfirst=True, parse_dates=['DateOfOrder'])

In [91]:
df1['OldData'] = np.where(df1['DateOfOrder'].values < np.datetime64(datetime(2021, 5, 1)), True, False)
# df1['OldData'] = df1['DateOfOrder'].apply(lambda x: True if x < datetime.strptime('01/05/2021', "%d/%m/%Y") else False)

In [92]:
# def mapping_amt_release(OldData,OpeiningOrderAmt,OrdAmtRelease):
#     if OldData == True:
#         return OpeiningOrderAmt
#     else:
#         return OrdAmtRelease
df1['OrdAmtRelease2'] = np.where(df1['OldData'].values, df1['OpeiningOrderAmt'].values, df1['OrdAmtRelease'].values)
df1['CountOrdRelease2'] = np.where(df1['OldData'].values, df1['CountOpeningOrder'].values, df1['CountOrdRelease'].values)
# df1['OrdAmtRelease2'] = df1.apply(lambda x: mapping_amt_release(x['OldData'], x['OpeiningOrderAmt'],x['OrdAmtRelease']),axis=1)
# df1['CountOrdRelease2'] = df1.apply(lambda x: mapping_amt_release(x['OldData'], x['CountOpeningOrder'],x['CountOrdRelease']),axis=1)
print(df1['OrdAmtRelease2'].sum())

183207942908.0


In [93]:
df1.columns

Index(['BranchID', 'SlsperID', 'Position', 'SlsName', 'PaymentsForm', 'Terms',
       'OrderDate', 'OrderNbr', 'DateOfOrder', 'CountOpeningOrder',
       'OpeiningOrderAmt', 'CountOrdRelease', 'OrdAmtRelease',
       'DeliveredOrder', 'DeliveredOrderAmt', 'CountReturnOrd', 'ReturnOrdAmt',
       'CountDebtConfirm', 'DebConfirmAmt', 'CountDebtConfirmRelease',
       'DebConfirmAmtRelease', 'OldData', 'OrdAmtRelease2',
       'CountOrdRelease2'],
      dtype='object')

In [94]:
import numpy as np # linear algebra
import pandas as pd

In [95]:
groupbylist = ['BranchID', 'Position', 'SlsperID', 'DateOfOrder','SlsName', 'Terms', 'PaymentsForm']

In [96]:
aggregate_dict = {
'OrdAmtRelease2':np.sum, 'DeliveredOrderAmt':np.sum, 'ReturnOrdAmt': np.sum,
'DebConfirmAmt': np.sum, 'DebConfirmAmtRelease': np.sum, 'CountReturnOrd': np.sum,
'CountOrdRelease2': np.sum, 'DeliveredOrder':np.sum,
'CountDebtConfirm': np.sum, 'CountDebtConfirmRelease':np.sum
}

In [97]:
df2 = df1.groupby(groupbylist).aggregate(aggregate_dict)
df2.head()

OrdAmtRelease2  \
BranchID Position SlsperID DateOfOrder SlsName          Terms PaymentsForm                   
MR0001   LOG      MR0025   2021-03-08  Phạm Ngọc Trường Debt  TM                       0.0   
                           2021-03-10  Phạm Ngọc Trường Debt  TM                  900008.0   
                           2021-03-11  Phạm Ngọc Trường Debt  CK                 1590000.0   
                           2021-03-12  Phạm Ngọc Trường Debt  TM                  900008.0   
                           2021-03-15  Phạm Ngọc Trường Debt  TM                 1200003.0   

                                                                            DeliveredOrderAmt  \
BranchID Position SlsperID DateOfOrder SlsName          Terms PaymentsForm                      
MR0001   LOG      MR0025   2021-03-08  Phạm Ngọc Trường Debt  TM                          0.0   
                           2021-03-10  Phạm Ngọc Trường Debt  TM                          0.0   
                           2021-03-11  Phạm Ngọc Trường Debt  CK                          0.0   
                           2021-03-12  Phạm Ngọc Trường Debt  TM                          0.0   
                           2021-03-15  Phạm Ngọc Trường Debt  TM                          0.0   

                                                                            ReturnOrdAmt  \
BranchID Position SlsperID DateOfOrder SlsName          Terms PaymentsForm                 
MR0001   LOG      MR0025   2021-03-08  Phạm Ngọc Trường Debt  TM                     0.0   
                           2021-03-10  Phạm Ngọc Trường Debt  TM                     0.0   
                           2021-03-11  Phạm Ngọc Trường Debt  CK                     0.0   
                           2021-03-12  Phạm Ngọc Trường Debt  TM                     0.0   
                           2021-03-15  Phạm Ngọc Trường Debt  TM                     0.0   

                                                                            DebConfirmAmt  \
BranchID Position SlsperID DateOfOrder SlsName          Terms PaymentsForm                  
MR0001   LOG      MR0025   2021-03-08  Phạm Ngọc Trường Debt  TM                2100011.0   
                           2021-03-10  Phạm Ngọc Trường Debt  TM                 900008.0   
                           2021-03-11  Phạm Ngọc Trường Debt  CK                1590000.0   
                           2021-03-12  Phạm Ngọc Trường Debt  TM                 900008.0   
                           2021-03-15  Phạm Ngọc Trường Debt  TM                1200003.0   

                                                                            DebConfirmAmtRelease  \
BranchID Position SlsperID DateOfOrder SlsName          Terms PaymentsForm                         
MR0001   LOG      MR0025   2021-03-08  Phạm Ngọc Trường Debt  TM                       2100011.0   
                           2021-03-10  Phạm Ngọc Trường Debt  TM                        900008.0   
                           2021-03-11  Phạm Ngọc Trường Debt  CK                       1590000.0   
                           2021-03-12  Phạm Ngọc Trường Debt  TM                        900008.0   
                           2021-03-15  Phạm Ngọc Trường Debt  TM                       1200003.0   

                                                                            CountReturnOrd  \
BranchID Position SlsperID DateOfOrder SlsName          Terms PaymentsForm                   
MR0001   LOG      MR0025   2021-03-08  Phạm Ngọc Trường Debt  TM                         0   
                           2021-03-10  Phạm Ngọc Trường Debt  TM                         0   
                           2021-03-11  Phạm Ngọc Trường Debt  CK                         0   
                           2021-03-12  Phạm Ngọc Trường Debt  TM                         0   
                           2021-03-15  Phạm Ngọc Trường Debt  TM                         0   

                                                                            Coun

In [98]:
df2 = df2.reset_index()

In [99]:
# https://datatofish.com/if-condition-in-pandas-dataframe/

In [100]:
df2.head()

,BranchID,Position,SlsperID,DateOfOrder,SlsName,Terms,PaymentsForm,OrdAmtRelease2,DeliveredOrderAmt,ReturnOrdAmt,DebConfirmAmt,DebConfirmAmtRelease,CountReturnOrd,CountOrdRelease2,DeliveredOrder,CountDebtConfirm,CountDebtConfirmRelease
0,MR0001,LOG,MR0025,2021-03-08,Phạm Ngọc Trường,Debt,TM,0.0,0.0,0.0,2100011.0,2100011.0,0,0,0,1,1
1,MR0001,LOG,MR0025,2021-03-10,Phạm Ngọc Trường,Debt,TM,900008.0,0.0,0.0,900008.0,900008.0,0,1,0,1,1
2,MR0001,LOG,MR0025,2021-03-11,Phạm Ngọc Trường,Debt,CK,1590000.0,0.0,0.0,1590000.0,1590000.0,0,1,0,1,1
3,MR0001,LOG,MR0025,2021-03-12,Phạm Ngọc Trường,Debt,TM,900008.0,0.0,0.0,900008.0,900008.0,0,1,0,1,1
4,MR0001,LOG,MR0025,2021-03-15,Phạm Ngọc Trường,Debt,TM,1200003.0,0.0,0.0,1200003.0,1200003.0,0,1,0,1,1


In [101]:
df2['tienchotso'] = np.where(df2['PaymentsForm'].values=='TM', df2['OrdAmtRelease2'].values, 0)
df2['tiengiaothanhcong'] = np.where(df2['PaymentsForm'].values=='TM', df2['DeliveredOrderAmt'].values, 0)
df2['tienhuydon'] = np.where(df2['PaymentsForm'].values=='TM', df2['ReturnOrdAmt'].values, 0)
df2['tienlenbangke'] = np.where(df2['PaymentsForm'].values=='TM', df2['DebConfirmAmt'].values, 0)
df2['tienthuquyxacnhan'] = np.where(df2['PaymentsForm'].values=='TM', df2['DebConfirmAmtRelease'].values, 0)
df2['chitietnocongty'] = df2['tienchotso'].values - df2['tienhuydon'].values - df2['tienthuquyxacnhan'].values
df2['tiennocongty'] = np.where(df2['chitietnocongty']>0, df2['chitietnocongty'].values, 0)
# df2['tienchotso'] = df2.apply(lambda x: x['OrdAmtRelease2'] if x['PaymentsForm']=='TM' else 0, axis=1)
# df2['tiengiaothanhcong'] = df2.apply(lambda x: x['DeliveredOrderAmt'] if x['PaymentsForm']=='TM' else 0, axis=1)
# df2['tienhuydon'] = df2.apply(lambda x: x['ReturnOrdAmt'] if x['PaymentsForm']=='TM' else 0, axis=1)
# df2['tienlenbangke'] = df2.apply(lambda x: x['DebConfirmAmt'] if x['PaymentsForm']=='TM' else 0, axis=1)
# df2['tienthuquyxacnhan'] = df2.apply(lambda x: x['DebConfirmAmtRelease'] if x['PaymentsForm']=='TM' else 0, axis=1)
# df2['chitietnocongty'] = df2['tienchotso'] - df2['tienhuydon'] - df2['tienthuquyxacnhan']
# df2['tiennocongty'] = df2.apply(lambda x: x['chitietnocongty'] if x['chitietnocongty']>0 else 0, axis=1)

In [102]:
rename_dict ={'CountOrdRelease2':'donchotso', 'DeliveredOrder':'dongiaothanhcong', 'CountReturnOrd':'donhuy',
'CountDebtConfirm':'donlenbangke', 'CountDebtConfirmRelease':'donthuquyxacnhan'
}

In [103]:
df2.rename(columns=rename_dict, inplace=True)

In [104]:
# df2['donchotso'] = df2.apply(lambda x: x['CountOrdRelease2'], axis=1)
# df2['dongiaothanhcong'] = df2.apply(lambda x: x['DeliveredOrder'] if x['PaymentsForm']=='TM' else 0, axis=1)
# df2['donlenbangke'] = df2.apply(lambda x: x['CountDebtConfirm'] if x['PaymentsForm']=='TM' else 0, axis=1)
# df2['donthuquyxacnhan'] = df2.apply(lambda x: x['CountDebtConfirmRelease'] if x['PaymentsForm']=='TM' else 0, axis=1)
df2['chitietdonnocongty'] = df2['donchotso'] - df2['donthuquyxacnhan'] - df2['donhuy']
df2['donnocongty'] = df2.apply(lambda x: x['chitietdonnocongty'] if x['chitietdonnocongty']>0 else 0, axis=1)

In [105]:
df2.columns

Index(['BranchID', 'Position', 'SlsperID', 'DateOfOrder', 'SlsName', 'Terms',
       'PaymentsForm', 'OrdAmtRelease2', 'DeliveredOrderAmt', 'ReturnOrdAmt',
       'DebConfirmAmt', 'DebConfirmAmtRelease', 'donhuy', 'donchotso',
       'dongiaothanhcong', 'donlenbangke', 'donthuquyxacnhan', 'tienchotso',
       'tiengiaothanhcong', 'tienhuydon', 'tienlenbangke', 'tienthuquyxacnhan',
       'chitietnocongty', 'tiennocongty', 'chitietdonnocongty', 'donnocongty'],
      dtype='object')

In [106]:
col_list = ['BranchID', 'Position', 'SlsperID', 'Ngày Tạo Đơn', 'MDS', 'Terms',
       'PaymentsForm', 'Sum of OrdAmtRelease', 'Sum of DeliveredOrderAmt',
       'Sum of ReturnOrdAmt', 'Sum of DebConfirmAmt',
       'Sum of DebConfirmAmtRelease', 'Don Huy', 'Đơn Chốt Sổ', 'Đơn Giao Thành Công',
       'Đơn Lên Bảng Kê', 'Đơn Thủ Quỹ Xác Nhận', 'Tiền Chốt Sổ',
       'Tiền Giao Thành Công', 'Tiền Hủy Đơn', 'Tiền Lên Bảng Kê',
       'Tiền Thủ Quỹ Xác Nhận', 'Chi Tiết Tiền Nợ Công Ty', 'Tiền Nợ Công Ty',
       'Chi Tiết Đơn Nợ Công Ty', 'Đơn Nợ Công Ty']

In [107]:
df2.columns = col_list
df2.to_csv('doanhthutienmat_data.csv', index=False)

In [108]:
# Google Sheet connection
scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive',
          'https://www.googleapis.com/auth/drive.file']
jsonfile = 'D:/data_sale/datateam1599968716114-6f9f144b4262.json'
credentials = service_account.Credentials.from_service_account_file(jsonfile, scopes=scopes)
service = discovery.build('drive', 'v3', credentials=credentials)

In [109]:
#spreadsheets_id = '1qEwviiJcAtvWCLvc-5AXOgidaeHMsi2RWWIKiUehabI'
folder_id = '12iN7KEKpgT4QhIfaLNB5shQ4g5x-VUoY'
file_name = 'doanhthutienmat_data.csv'
file_type = 'text/csv'
file_body={'name':file_name,'addParents':[folder_id]}
query = f"parents = '{folder_id}'"
files = service.files().list(q=query).execute()
fileid  = files['files'][0]['id']

media = MediaFileUpload('{0}'.format(file_name), mimetype=file_type)
updated_file = service.files().update(fileId=fileid, body=file_body, media_body=media)
updated_file.execute()

{'kind': 'drive#file',
 'id': '1TUYHJ_xqTU_L57TOsBKllgjWXF_ZkMkn',
 'name': 'doanhthutienmat_data.csv',
 'mimeType': 'text/csv'}